# 转录因子活性推断

使用 SCENIC 进行转录因子活性推断

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import exprmat as em

# set working directory
em.setwd('/home/data/yangz/shared/tutorial/single-cell-rna')

In [3]:
expm = em.load_experiment('expression', load_samples = True, load_subset = 'mono-neutro')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━ loading samples          4 / 4     (00:05 < 00:00)


In [4]:
print(expm)

annotated data of size 10160 × 19389
subset mono-neutro of size 10160 × 19389
contains modalities: rna

 modality [rna]
    obs : sample <cat> batch <cat> group <cat> modality <cat> taxa <cat> barcode <o> ubc <o>
          n.umi <f64> n.genes <i64> n.mito <f64> n.ribo <f64> pct.mito <f64> pct.ribo <f64>
          filter <bool> score.doublet <f64> score.doublet.se <f64> is.doublet <bool> qc <bool>
          leiden <cat> cell.type <cat> score <f64> potency <cat> relative <f64> score.preknn <f64>
          potency.preknn <cat> ppt.pseudotime <f64> ppt.seg <cat> ppt.edge <cat>
          ppt.milestones <cat>
    var : chr <cat> start <i64> end <i64> strand <cat> id <o> subtype <cat> gene <cat> tlen <f64>
          cdslen <i64> assembly <cat> uid <o> vst.hvg <bool> vst.all.means <f64> vst.all.vars <f64>
          vst.all.vars.norm <f64> vst.all.hvg.rank <f32> vst.all.hvg <bool> n.umi <f64> symbol <cat>
 layers : counts <f32> magic <f64> norm <f32>
   obsm : cnmf.10 <df> diffmap <arr:f32(5)> 

### SCENIC 分析

为了减小计算量并增大稳定性，我们在元细胞上计算 SCENIC

In [30]:
expm.modalities['rna']['metacell'].layers['counts'] = em.np.expm1(
    expm.modalities['rna']['metacell'].X
).floor()

In [7]:
expm.run_rna_infer_tf_activity(
    run_on_samples = ['metacell'],

    layer = 'counts',
    gene = 'gene',
    method = 'grnboost2',
    taxa = 'mmu',
    ncpus = 100,
    seed = 42,

    features = 'genes',
    cistromes = 'motifs',
    thresholds = [0.75, 0.90],
    top_n_targets = [50],
    top_n_regulators = [5, 10, 50],
    min_genes = 20,
    mask_dropout = True,
    keep_only_activating = True,
    rank_threshold = 5000,
    auc_threshold = 0.05,
    nes_threshold = 3.0,
    max_similarity_fdr = 0.001,
    min_orthologous_identity = 0,
    chunk_size = 100,

    weighted = False,

    key_adjacencies = 'tf.adjacencies',
    key_added = 'tf'
)

[i] loaded 1860 transcriptional factors.
[i] starting grnboost2 using 80 processes


   ━━━━━━━━━━━━━━━━━━━ inferring partial network 19078 / 19078 (1:10:46 < 00:00)


[i] finished in 4249.272406101227 seconds.


[i] calculating pearson correlations ...
[i] using 100 workers.
[i] creating regulons from a dataframe of enriched features.
[i] additional columns saved: []


   ━━━━━━━━━━━━━━━━━━━━━━━━━━ auc                    236 / 236   (00:05 < 00:00)


In [8]:
expm.modalities['rna']['metacell'].uns['tf.adjacencies']

,tf,target,importance
450,Anxa1,S100a8,2.318543e+01
450,Anxa1,S100a9,2.195228e+01
442,Prdx5,S100a8,2.173934e+01
442,Prdx5,S100a9,2.107148e+01
500,Zeb2,S100a8,1.612040e+01
...,...,...,...
769,Abcf2,Rsph9,2.961198e-19
145,Xrcc4,Hbb-bt,2.957419e-19
627,Dhx36,Has1,4.824099e-20
1218,Pkm,Has1,1.101952e-20


In [10]:
expm.modalities['rna']['metacell'].obsm['tf'].iloc[:, 0:6]

regulon,Ahdc1 (+),Ahr (+),Arid3a (+),Arnt (+),Atf1 (+),Atf3 (+)
mc:integrated:1,0.091195,0.039358,0.027912,0.0,0.050292,0.068105
mc:integrated:2,0.000000,0.038827,0.034006,0.0,0.004394,0.109103
mc:integrated:3,0.000000,0.048634,0.045927,0.0,0.034814,0.069670
mc:integrated:4,0.000000,0.040191,0.036249,0.0,0.030777,0.045177
mc:integrated:5,0.000000,0.020117,0.039077,0.0,0.010616,0.043169
...,...,...,...,...,...,...
mc:integrated:1380,0.008386,0.050358,0.026132,0.0,0.062848,0.102932
mc:integrated:1381,0.068284,0.025631,0.029009,0.0,0.011330,0.099359
mc:integrated:1382,0.000000,0.040364,0.028375,0.0,0.015188,0.096717
mc:integrated:1383,0.000000,0.045030,0.037443,0.0,0.014987,0.037382


In [11]:
em.memory()

[i] resident memory: 4.60 GiB
[i] virtual memory: 20.18 GiB


In [12]:
expm.save()

[i] main dataset write to expression/subsets/mono-neutro.h5mu
[i] saving individual samples. (pass `save_samples = False` to skip)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━ modality [rna]           4 / 4     (00:02 < 00:00)
